# Введение

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.
Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Наша задача  — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

###### Описание данных

     Таблица books
   Содержит данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

      Таблица authors
Содержит данные об авторах:
- author_id — идентификатор автора;
- author — имя автора.

      Таблица publishers
Содержит данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства;

      Таблица ratings
Содержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

      Таблица reviews
Содержит данные о пользовательских обзорах:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- text — текст обзора.

###### В ходе анализа ответим на следующие вопросы:

1. Посчитаем, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитаем количество обзоров и среднюю оценку;
3. Определим издательство, которое выпустило наибольшее число книг толще 50 страниц;
4. Определим автора с самой высокой средней оценкой книг среди книг с 50 и более оценками;
5. Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

## Подгрузка базы данных и знакомство с таблицами

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine


In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * 
           FROM books'''
books = pd.io.sql.read_sql(query, con = engine)
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
query = '''SELECT * 
           FROM authors'''
authors = pd.io.sql.read_sql(query, con = engine)
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
query = '''SELECT * 
           FROM ratings'''
ratings = pd.io.sql.read_sql(query, con = engine)
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
query = '''SELECT * 
           FROM reviews'''
reviews = pd.io.sql.read_sql(query, con = engine)
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [7]:
query = '''SELECT * 
           FROM publishers'''
publishers = pd.io.sql.read_sql(query, con = engine)
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [8]:
# посмотрим информацию обо всех таблицах
data_list = (books, authors, publishers, ratings, reviews)

for table in data_list:
    table.info()
    print()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------      

Познакомились с таблицами, выяснили, что:
1. В таблице books 1000 строк
2. В таблице authors 636 строк
3. В таблице publishers 340 строк
3. В таблице ratings 6456 строк
3. В таблице reveiws 2793 строки

## Выполним предобработку данных

In [9]:
#проверим количество пропусков в таблицах
for table in data_list:
    print('______________________________________________________________')
    print(table.isna().sum().reset_index())
    print('______________________________________________________________')

______________________________________________________________
              index  0
0           book_id  0
1         author_id  0
2             title  0
3         num_pages  0
4  publication_date  0
5      publisher_id  0
______________________________________________________________
______________________________________________________________
       index  0
0  author_id  0
1     author  0
______________________________________________________________
______________________________________________________________
          index  0
0  publisher_id  0
1     publisher  0
______________________________________________________________
______________________________________________________________
       index  0
0  rating_id  0
1    book_id  0
2   username  0
3     rating  0
______________________________________________________________
______________________________________________________________
       index  0
0  review_id  0
1    book_id  0
2   username  0
3       text  0
_______

In [10]:
#проверим количество дубликатов в таблицах
for table in data_list:
    print('______________________________________________________________')
    print(table.duplicated().sum())
    print('______________________________________________________________')

______________________________________________________________
0
______________________________________________________________
______________________________________________________________
0
______________________________________________________________
______________________________________________________________
0
______________________________________________________________
______________________________________________________________
0
______________________________________________________________
______________________________________________________________
0
______________________________________________________________


###### Вывод

В данных отсутствуют пропуски и дубликаты, данные готовы для решения поставленных задач.

## Поиск ответов на поставленные вопросы

In [11]:
#пробуем создать функцию для запросов к БД
def sql(sql):
    return pd.read_sql(sql, con=engine)

### Посчитаем, сколько книг вышло после 1 января 2000 года

In [12]:
q = '''SELECT COUNT(book_id) AS count
           FROM books
           WHERE publication_date >= '2000-01-01'
           '''
sql(q)

,count
0,821


- После 1 января 2000 года вышла 821 книга

### Для каждой книги посчитаем количество обзоров и среднюю оценку

In [17]:
q = '''SELECT b.book_id,
              b.title,
              AVG(r.rating) AS avg_rating,
              COUNT(DISTINCT rv.review_id) AS count_reviews
       FROM books AS b
       LEFT JOIN ratings AS r ON b.book_id = r.book_id
       LEFT JOIN reviews AS rv ON b.book_id = rv.book_id
       GROUP BY b.book_id;
'''

sql(q)

,book_id,title,avg_rating,count_reviews
0,1,'Salem's Lot,3.666667,2
1,2,1 000 Places to See Before You Die,2.500000,1
2,3,13 Little Blue Envelopes (Little Blue Envelope...,4.666667,3
3,4,1491: New Revelations of the Americas Before C...,4.500000,2
4,5,1776,4.000000,4
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3.666667,3
996,997,Xenocide (Ender's Saga #3),3.400000,3
997,998,Year of Wonders,3.200000,4
998,999,You Suck (A Love Story #2),4.500000,2


### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц

In [14]:
q = '''SELECT p.publisher,
              COUNT (b.book_id)
       FROM publishers AS p
       JOIN books AS b ON b.publisher_id = p.publisher_id
       WHERE b.num_pages > 50
       GROUP BY p.publisher_id
       ORDER BY COUNT(b.book_id) DESC
       LIMIT 1;
'''

sql(q)

,publisher,count
0,Penguin Books,42


- Наибольшее число книг толще 50 страниц выпустило издательство Penguin Books, которое выпустило 42 книги.

### Определим автора с самой высокой средней оценкой книги среди книг с 50 и более оценками

In [15]:
q = '''     
     SELECT a.author,
            AVG(r.rating) avg_rating
     FROM authors AS a
     JOIN books AS b ON a.author_id=b.author_id
     JOIN ratings AS r ON b.book_id=r.book_id
     WHERE b.book_id IN 
                     (SELECT b.book_id
                      FROM books AS b
                      JOIN ratings AS r ON b.book_id=r.book_id
                      GROUP BY b.book_id
                      HAVING COUNT(r.rating_id)>=50)
     GROUP BY a.author
     ORDER BY avg_rating DESC
     LIMIT 1
'''

sql(q)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097


- Автор с самой  высокой средней оценкой книги - J.K. Rowling/Mary GrandPré	и средний рейтинг = 4.3

### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [16]:
q = ''' SELECT AVG(rev.rev_count) AS avg_review
        FROM
            (SELECT COUNT(review_id) AS rev_count
             FROM reviews
             WHERE username IN 
                              (SELECT username
                               FROM ratings
                               GROUP BY username
                               HAVING COUNT(ratings) > 48)
             GROUP BY username) AS rev;    
    
'''

sql(q)

,avg_review
0,24.0


- Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24

## Вывод

Проведя работу по анализу предоставленной базы данных нашли ответы на поставленные вопросы:
1. После 1 января 2000 года вышла 821 книга
2. Составили таблицу с количеством обзоров и средней оценкой для каждой книги
3. Наибольшее число книг толще 50 страниц выпустило издательство Penguin Books, которое выпустило 42 книги

